In [ ]:
import joblib
import pandas as pd
import xgboost
import shap

LABEL_COLUMN = "churned"

shap.initjs() 

# Load dataset

In [ ]:
data = pd.read_csv('../data/users_train.csv')
x_test = data.drop(columns=LABEL_COLUMN)
y_test = data[LABEL_COLUMN]

# Load best model

In [ ]:
pipeline = joblib.load('models/xgb_model_full.joblib')

In [ ]:
preprocesor = pipeline['preprocessor']
model = pipeline['classifier']

In [ ]:
# Prepare dataset

In [ ]:
observations = preprocesor.transform(x_test)

In [ ]:
column_name = (
    preprocesor.transformers_[0][2]
    + list(preprocesor.transformers_[1][1]['one-hot'].get_feature_names(preprocesor.transformers[1][2]))
)

observations_df = pd.DataFrame(observations, columns=column_name)

In [ ]:
observations_df

# Interpretability

## Feature importance desde el modelo 

In [ ]:
(
    pd.Series(model.feature_importances_, index=observations_df.columns)
    .sort_values(ascending=False)
    .iloc[:10]
    .plot(kind="bar")
)

## Shap Values

### TreeExplainer

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(observations)

In [ ]:
shap.summary_plot(shap_values, observations_df, plot_type="bar")

### Explainer

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer(observations_df)

In [ ]:
shap_values

#### visualize the first prediction's explanation

In [ ]:

shap.plots.waterfall(shap_values[0])

#### visualize all predictiosn

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
shap.plots.force(shap_values[0])

In [ ]:
#shap.plots.force(explainer.expected_value, shap_values.values)